In [18]:
# ---------------------------------------------------------------------------------
# Initializing RF program
# ---------------------------------------------------------------------------------
print('---------------------------------------------------------------------------------')
print('Initializing RF program')
print('---------------------------------------------------------------------------------\n')

# ---------------------------------------------------------------------------------
# Importing Libraries
# ---------------------------------------------------------------------------------
print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------\n')

import time
import os
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, label_binarize
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_curve, roc_auc_score, auc
from sklearn.model_selection import train_test_split

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

import shap
from scipy.special import softmax

# ✅ PyTorch (we don’t build an NN here, but you asked to migrate from TF → PyTorch)
import torch
print("PyTorch version:", torch.__version__)

np.random.seed(0)


---------------------------------------------------------------------------------
Initializing RF program
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------

PyTorch version: 2.6.0+cu124


In [19]:
# ---------------------------------------------------------------------------------
# Defining Metric Equations
# ---------------------------------------------------------------------------------
print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------\n')

def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse=True)}
    feature_importances_norm = {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse=True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

def ACC(TP, TN, FP, FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc

def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac

def PRECISION(TP, FP):
    Precision = TP/(TP+FP)
    return Precision

def RECALL(TP, FN):
    Recall = TP/(TP+FN)
    return Recall

def F1_score(Recall, Precision):  # renamed to avoid variable/function collision later
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1

def BACC(TP, TN, FP, FN):
    BACC_val = (TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC_val

def MCC(TP, TN, FP, FN):
    MCC_val = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC_val

def AUC_ROC(y_test_bin, y_score):
    auc_avg = 0
    counting = 0
    for i in range(y_test_bin.shape[1]):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        auc_avg += auc(fpr, tpr)
        counting = i+1
    return auc_avg/counting


---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------



In [20]:
# ---------------------------------------------------------------------------------
# Defining features of interest
# ---------------------------------------------------------------------------------
print('---------------------------------------------------------------------------------')
print('Defining features of interest')
print('---------------------------------------------------------------------------------\n')

# Keep only the final req_cols version you used
req_cols = [
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',' Label'
]

# ---------------------------------------------------------------------------------
# Loading Databases
# ---------------------------------------------------------------------------------
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------\n')

files = [
    'Wednesday-workingHours.pcap_ISCX.csv',
    'Tuesday-WorkingHours.pcap_ISCX.csv',
    'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    'Monday-WorkingHours.pcap_ISCX.csv',
    'Friday-WorkingHours-Morning.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
]

frames = [pd.read_csv(file, usecols=req_cols) for file in files]
df = pd.concat(frames, ignore_index=True)
df = df.sample(frac=1)
print("Loaded shape:", df.shape)


---------------------------------------------------------------------------------
Defining features of interest
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

Loaded shape: (2830743, 16)


In [21]:
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------\n')

df_max_scaled = df.copy()
y = df_max_scaled[' Label'].replace({
    'DoS GoldenEye': 'Dos/Ddos',
    'DoS Hulk': 'Dos/Ddos',
    'DoS Slowhttptest': 'Dos/Ddos',
    'DoS slowloris': 'Dos/Ddos',
    'Heartbleed': 'Dos/Ddos',
    'DDoS': 'Dos/Ddos',
    'FTP-Patator': 'Brute Force',
    'SSH-Patator': 'Brute Force',
    'Web Attack - Brute Force': 'Web Attack',
    'Web Attack - Sql Injection': 'Web Attack',
    'Web Attack - XSS': 'Web Attack',
    'Web Attack XSS': 'Web Attack',
    'Web Attack Sql Injection': 'Web Attack',
    'Web Attack Brute Force': 'Web Attack'
})
df_max_scaled.pop(' Label')

for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col] / t

df = df_max_scaled.assign(Label=y)
df = df.fillna(0)
print("After normalization:", df.shape)


---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------

After normalization: (2830743, 16)


In [22]:
print('---------------------------------------------------------------------------------')
print('Counting labels')
print('---------------------------------------------------------------------------------\n')

y = df.pop('Label')
X = df
counter = Counter(y)
print(counter)

df = X.assign(Label=y)
df = df.drop_duplicates()
y = df.pop('Label')
X = df
print('after removing duplicates:', Counter(y))
df = X.assign(Label=y)

print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------\n')

df['is_train'] = np.random.uniform(0, 1, len(df)) <= .70

train, test = df[df['is_train']==True], df[df['is_train']==False]
features = df.columns[:len(req_cols)-1]

y_train, label = pd.factorize(train['Label'])
y_test, label1 = pd.factorize(test['Label'])

print("Train size:", train.shape, "| Test size:", test.shape)


---------------------------------------------------------------------------------
Counting labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 2273097, 'Dos/Ddos': 380699, 'PortScan': 158930, 'Brute Force': 13835, 'Bot': 1966, 'Web Attack � Brute Force': 1507, 'Web Attack � XSS': 652, 'Infiltration': 36, 'Web Attack � Sql Injection': 21})
after removing duplicates: Counter({'BENIGN': 848620, 'Dos/Ddos': 73395, 'PortScan': 2237, 'Bot': 785, 'Brute Force': 707, 'Web Attack � Brute Force': 91, 'Infiltration': 36, 'Web Attack � XSS': 32, 'Web Attack � Sql Injection': 17})
---------------------------------------------------------------------------------
Separating Training and Testing db
---------------------------------------------------------------------------------

Train size: (648526, 17) | Test size: (277394, 17)


In [23]:
print('---------------------------------------------------------------------------------')
print('Defining the RF model')
print('---------------------------------------------------------------------------------\n')

clf = RandomForestClassifier(max_depth=10, n_estimators=100, min_samples_split=2, n_jobs=-1)

print('---------------------------------------------------------------------------------')
print('Training the model')
print('---------------------------------------------------------------------------------\n')

start = time.time()
model = clf.fit(train[features], y_train)
print('ELAPSE TIME MODEL: ', (time.time() - start)/60, 'min')
print('------------------------------------------------------------------------------')

start = time.time()
preds = clf.predict(test[features])
print('ELAPSE TIME PREDICTION: ', (time.time() - start)/60, 'min')

y_pred = clf.predict_proba(test[features])
pred_label = label[preds]


---------------------------------------------------------------------------------
Defining the RF model
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Training the model
---------------------------------------------------------------------------------

ELAPSE TIME MODEL:  2.164467533429464 min
------------------------------------------------------------------------------
ELAPSE TIME PREDICTION:  0.02822315295537313 min


In [24]:
print('---------------------------------------------------------------------------------')
print('Generating Confusion Matrix')
print('---------------------------------------------------------------------------------\n')

confusion_matrix = pd.crosstab(
    test['Label'], pred_label,
    rownames=['Actual ALERT'], colnames=['Predicted ALERT'],
    dropna=False
).sort_index(axis=0).sort_index(axis=1)

all_unique_values = sorted(set(pred_label) | set(test['Label']))
z = np.zeros((len(all_unique_values), len(all_unique_values)))
rows, cols = confusion_matrix.shape
z[:rows, :cols] = confusion_matrix
confusion_matrix = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
print(confusion_matrix)

FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)

TP_total = np.array(sum(TP), dtype=np.float64)
TN_total = np.array(sum(TN), dtype=np.float64)
FP_total = np.array(sum(FP), dtype=np.float64)
FN_total = np.array(sum(FN), dtype=np.float64)

Acc = ACC(TP_total, TN_total, FP_total, FN_total)
Precision = PRECISION(TP_total, FP_total)
Recall = RECALL(TP_total, FN_total)
F1_total = F1_score(Recall, Precision)
BACC_total = BACC(TP_total, TN_total, FP_total, FN_total)
MCC_total = MCC(TP_total, TN_total, FP_total, FN_total)

print('---------------------------------------------------------------------------------')
print('Accuracy total: ', Acc)
print('Precision total: ', Precision )
print('Recall total: ', Recall )
print('F1 total: ', F1_total )
print('BACC total: ', BACC_total)
print('MCC total: ', MCC_total)

y_test_bin = label_binarize(y_test, classes=list(range(len(label))))
n_classes = y_test_bin.shape[1]
print('AUC_ROC total: ', AUC_ROC(y_test_bin, y_pred))

for i in range(0, len(TP)):
    Acc_i = ACC(TP[i], TN[i], FP[i], FN[i])
    print('Accuracy: ', label[i] ,' - ' , Acc_i)
print('---------------------------------------------------------------------------------')


---------------------------------------------------------------------------------
Generating Confusion Matrix
---------------------------------------------------------------------------------

                              BENIGN   Bot  Brute Force  Dos/Ddos  \
BENIGN                      254298.0   0.0          0.0       3.0   
Bot                            254.0  18.0          0.0       0.0   
Brute Force                      9.0   0.0        207.0       0.0   
Dos/Ddos                       228.0   0.0          0.0   21449.0   
Infiltration                    14.0   0.0          0.0       0.0   
PortScan                       181.0   0.0          0.0       0.0   
Web Attack � Brute Force         8.0   0.0          0.0       0.0   
Web Attack � Sql Injection       5.0   0.0          0.0       0.0   
Web Attack � XSS                 6.0   0.0          0.0       0.0   

                            Infiltration  PortScan  Web Attack � Brute Force  \
BENIGN                              

/tmp/ipython-input-3218138353.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Acc_i = ACC(TP[i], TN[i], FP[i], FN[i])


In [25]:
# Drop helper columns before SHAP viz
test.pop('Label')
test.pop('is_train')

plt.clf()
# ## Summary Bar Plot Global
explainer = shap.TreeExplainer(clf)
start_index = 0
end_index = min(500, len(test))
shap_values = explainer.shap_values(test[start_index:end_index])
shap_obj = explainer(test[start_index:end_index])

shap.summary_plot(
    shap_values=shap_values,
    features=test[start_index:end_index],
    class_names=[label[i] for i in range(min(7, len(label)))],
    show=False
)
plt.savefig('RF_Shap_Summary_global_cicids.png')
plt.clf()

vals = np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(
    list(zip(train.columns, sum(vals))),
    columns=['col_name','feature_importance_vals']
)
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
print(feature_importance.head(20))

# ## Summary Dot Plot (Global)
shap.summary_plot(
    shap_values=np.take(shap_obj.values, 0, axis=-1),
    features=test[start_index:end_index],
    show=False
)
plt.savefig('RF_Shap_Summary_Beeswarms_cicids.png')
plt.clf()


/tmp/ipython-input-2635521019.py:13: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(
/usr/local/lib/python3.11/dist-packages/shap/plots/_beeswarm.py:723: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  summary_legacy(
/usr/local/lib/python3.11/dist-packages/shap/plots/_beeswarm.py:743: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  summary_legacy(
/usr/local/lib/python3.11/dist-packages/shap/plots/_beeswarm.py:743: FutureWarning: The NumPy gl

                       col_name  feature_importance_vals
0              Destination Port                 7.377458
2   Total Length of Bwd Packets                 5.776469
1   Total Length of Fwd Packets                 2.490665
4        Bwd Packet Length Mean                 0.579661
6            Packet Length Mean                 0.297194
3         Bwd Packet Length Max                 0.227924
7             Packet Length Std                 0.176619
5             Max Packet Length                 0.098193
8        Packet Length Variance                 0.018460


/tmp/ipython-input-2635521019.py:31: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(


<Figure size 640x480 with 0 Axes>

<Figure size 800x750 with 0 Axes>